# Fractional Property ownership

In [1]:
from web3.auto import w3
import requests
import json
import os
from dotenv import load_dotenv
from pathlib import Path

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


In [2]:
load_dotenv()

def initContract(contract_address: str, contract_abi_path: str):
    with open(Path(contract_abi_path)) as json_file:
        abi = json.load(json_file)
        
    if (contract_address.find(";") > -1 or contract_address.find(";") > -1):
        contract_address = contract_address.replace(";", "").replace("'", "")

    return w3.eth.contract(address=contract_address, abi=abi)

def convertDataToJSON(content: dict):
    data = {
        "pinataOptions": {"cidVersion": 1},
        "pinataContent": content
    }
    return json.dumps(data)

def pinJSONtoIPFS(json):
    pinata_api_key = os.getenv("PINATA_API_KEY")

    if (pinata_api_key.find(";") > -1 or pinata_api_key.find(";") > -1):
        pinata_api_key = pinata_api_key.replace(";", "").replace("'", "")

    pinata_secret_api_key = os.getenv("PINATA_SECRET_API_KEY")

    if (pinata_secret_api_key.find(";") > -1 or pinata_secret_api_key.find(";") > -1):
        pinata_secret_api_key = pinata_secret_api_key.replace(";", "").replace("'", "")

    headers = {
        "Content-Type": "application/json",
        "pinata_api_key": pinata_api_key,
        "pinata_secret_api_key": pinata_secret_api_key
    }

    r = requests.post("https://api.pinata.cloud/pinning/pinJSONToIPFS", data=json, headers=headers)
    # print(r.json())
    ipfs_hash = r.json()["IpfsHash"]
    return f"ipfs://{ipfs_hash}"

In [3]:
def register_user(user_info: dict):    
    wallet = user_info['wallet']        

    data = convertDataToJSON(user_info)
    user_info_URI = pinJSONtoIPFS(data)
    
    tx_hash = RegistrationContract.functions.registerUser(wallet, user_info_URI).transact({
        "from": w3.eth.accounts[0]
    })
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)    

    return receipt  

def register_property(property_info: dict):
    owner_wallet = property_info['owner_wallet']    

    data = convertDataToJSON(property_info)
    property_URI = pinJSONtoIPFS(data)
    
    tx_hash = RegistrationContract.functions.registerProperty(owner_wallet, property_URI).transact({
        "from": w3.eth.accounts[0]
    })
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    # print(receipt)

    # processed_receipt = RegistrationContract.events.RegisterProperty().processReceipt(receipt)
    # print(processed_receipt)

    # output = f"userID: {processed_receipt[0].args.userID}"
    # print(output)

    return receipt

def register_property_management(property_management_info: dict):
    wallet = property_management_info['wallet'] 

    data = convertDataToJSON(property_management_info)
    property_management_URI = pinJSONtoIPFS(data)

    tx_hash = RegistrationContract.functions.registerPropertyManagement(wallet, property_management_URI).transact({
        "from": w3.eth.accounts[0]
    })
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    return receipt            

# def buy_token(from_address: str, to_address:str, tokens_to_purchase:int):
    # pass

# def get_balance(address):
    # pass

## Init. smart contracts

In [4]:
RegistrationContract = initContract(os.getenv('REGISTRATION_CONTRACT_ADDRESS'), "./contracts/registration_contract_abi.json")

## Register owner for a property

In [5]:
owner_full_name = input("Please enter the owner's full name: ")    
owner_wallet = input("Please enter the owner's wallet address: ")

register_user(
    user_info={
        "wallet": owner_wallet, 
        "full_name": owner_full_name    
    }) 

AttributeDict({'transactionHash': HexBytes('0x5e69d530fc4fb7afa69cbaa0797e21b8815ae33d37d949e1354f21634fe4696a'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xe16c913524c8d59594911c6381de598c6d24efc3705803b8bfa7cbfdfa63de2b'),
 'blockNumber': 31,
 'from': '0xDFFc2C9a8F9408B7Fd9D3998cf6c3f6b1cB9b520',
 'to': '0xd8b934580fcE35a11B58C6D73aDeE468a2833fa8',
 'gasUsed': 22888,
 'cumulativeGasUsed': 22888,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

## Register Investor 1

In [ ]:
investor1_full_name = input("Please enter the first investor's full name: ")    
investor1_wallet = input("Please enter the first investor's wallet address: ")

register_user(
    user_info={
        "wallet": investor1_wallet, 
        "full_name": investor1_full_name    
    })

## Register Investor 2

In [ ]:
investor2_full_name = input("Please enter the second investor's full name: ")    
investor2_wallet = input("Please enter the second investor's wallet address: ")

register_user(        
    user_info={
        "wallet": investor2_wallet, 
        "full_name": investor2_full_name    
    })

## Register Investor 3

In [ ]:
investor3_full_name = input("Please enter the third investor's full name: ")    
investor3_wallet = input("Please enter the third investor's wallet address: ")

register_user(        
    user_info={
        "wallet": investor3_wallet, 
        "full_name": investor3_full_name    
    })   

## Register property (from the above owner)

In [ ]:
property_street_address = input("Please enter the property's street address: ")
property_city = input("Please enter the property's city: ")
property_state = input("Please enter the property's state: ")    
property_zip_code = input("Please enter the property's zip code: ")
property_validation_id = input("Please enter the property's validation id: ")    
property_price = input("Please enter the property's selling price: ")    
property_minted_tokens = input("Please enter the number of token to be minted for the property: ")    
property_auction_date = input("Please enter the auction date: ")

property_id = register_property(                
    property_info={
        "owner_wallet": owner_wallet, 
        "address": {
            "street_address": property_street_address,
            "city": property_city,
            "state": property_state,
            "zip_code": property_zip_code
        },
        "auction_date": property_auction_date,
        "price": property_price,
        "minted_tokens": property_minted_tokens,
        "validation_id": property_validation_id
    })

## Investor 1 buys tokens for property

In [ ]:
# buy_token(from: 'owner_wallet', to: 'address', amt_of_token:0)

## Investor 2 buys tokens for property

In [ ]:
# buy_token(from: 'owner_wallet', to: 'address', amt_of_token:0)

## Investor 3 buys tokens for property

In [ ]:
# buy_token(from: 'owner_wallet', to: 'address', amt_of_token:0)

## Investor 1 gets balance of token purchased

In [16]:
# get_balance(address)

## Investor 2 gets balance of token purchased

In [31]:
# get_balance(address)

## Investor 3 gets balance of token purchased

In [32]:
# get_balance(address)

## Pay rent

In [19]:
# Add user (renter)
# transfer tokens from renter to property's wallet

## Distribute profits from rent paid

In [ ]:
# profit splitter is executed on the property's wallet and distribute investor based on tokens purchased. 